<h1 style="text-align: center;">MODELE xCxF</h1>

<img 
    style="display: block; 
           margin-left: auto;
           margin-right: auto;"
    src="/Users/charley.dlebarbier/Documents/Dev/VSCode/Projects/projet_NeoAIssyr/extra/model_xcxf.png" 
    alt="model_xcxf">
</img>

# -- SETUP THE ENVIRONMENT

In [3]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [2]:
%pip install -q dagshub mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.2/211.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 17.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.4/238.4 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7

In [4]:
# -- General Import
import dagshub
import keras
import matplotlib.pyplot as plt
import mlflow
import numpy as np
import tensorflow as tf

# -- Keras
from keras.layers import Conv2D, Dense, Dropout, BatchNormalization,\
                         MaxPooling2D, ReLU, Rescaling
from keras.metrics import Accuracy, F1Score, Precision, Recall,\
                          TopKCategoricalAccuracy
from keras.models import Sequential
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import image_dataset_from_directory, to_categorical

# -- Other
from sklearn.metrics import classification_report, confusion_matrix

In [5]:
## - Prepare Dagshub and MLFlow tracking
dagshub.init(repo_owner='CharleyDL', repo_name='neo_aissyr', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=454466d9-ab2d-47dd-94a3-bb5f9f26ff5f&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=64890c2d2cd78a571d5c9e8737e3ef7400d82a4d1e576b1493f06da29aeb77fc




Output()

Initialized MLflow to track repo "CharleyDL/neo_aissyr"

Repository CharleyDL/neo_aissyr initialized!

# -- LOADING DATASET

In [6]:
directory_full = "/content/drive/MyDrive/projet_Assyr/training_data/glyphs_image"

full_ds = image_dataset_from_directory(
    directory_full,
    labels='inferred',
    label_mode='categorical',
    # class_names='inferred',

    subset='both',
    validation_split=0.2,

    image_size=(30,30),
    batch_size=64,
    color_mode='grayscale',

    seed=42
)

Found 7876 files belonging to 284 classes.
Using 6301 files for training.
Using 1575 files for validation.


In [7]:
class_names = full_ds[0].class_names
len(class_names)

284

# -- VISUALIZATION DATA

##### └── Train

In [ ]:
plt.figure(figsize=(10, 10))

for images, labels in full_ds[0].take(1):
    ## - convert one-hot encoder label to integer label
    integer_labels = np.argmax(labels, axis=1)

    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.title(class_names[integer_labels[i]])
        plt.axis("off")
        plt.imshow(images[i].numpy().astype("uint8"))

##### └── Validation

In [ ]:
plt.figure(figsize=(10, 10))

for images, labels in full_ds[1].take(1):
    ## - convert one-hot encoder label to integer label
    integer_labels = np.argmax(labels, axis=1)

    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.title(class_names[integer_labels[i]])
        plt.axis("off")
        plt.imshow(images[i].numpy().astype("uint8"))

# -- PREPROCESSING DATA

In [ ]:
## - Normalization
full_ds[0] = full_ds[0].map(lambda x, y: (Rescaling(1./255)(x), y)) # train
full_ds[1] = full_ds[1].map(lambda x, y: (Rescaling(1./255)(x), y)) # test

## - Check the normalization
print("Train Set:")
for image, label in full_ds[0].take(1):
    print("Minimum pixel value:", tf.reduce_min(image))
    print("Maximum pixel value:", tf.reduce_max(image))

print("\nValidation Set:")
for image, label in full_ds[1].take(1):
    print("Minimum pixel value:", tf.reduce_min(image))
    print("Maximum pixel value:", tf.reduce_max(image))

In [ ]:
## - Buffered Prefetching
AUTOTUNE = tf.data.AUTOTUNE

full_ds[0] = full_ds[0].cache().prefetch(buffer_size=AUTOTUNE)
full_ds[1] = full_ds[1].cache().prefetch(buffer_size=AUTOTUNE)

# -- DATA AUGMENTATION
-> compenser le déséquilibre des classes avec peu de données

In [ ]:
## Data Aug

# -- MODEL 8C2F

In [ ]:
model_8c2f = tf.keras.models.Sequential([
    ## - First Block
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), strides=2,
                           padding='same', input_shape=(30, 30, 1)),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), strides=2,
                           padding='same', input_shape=(30, 30, 1)),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2, padding='same'),
    tf.keras.layers.Dropout(0.5),

    ## - Second Block
    tf.keras.layers.Conv2D(64, kernel_size=(3, 3), strides=2,
                           padding='same', input_shape=(15, 15, 1)),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv2D(64, kernel_size=(3, 3), strides=2,
                           padding='same', input_shape=(15, 15, 1)),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv2D(64, kernel_size=(3, 3), strides=2,
                           padding='same', input_shape=(15, 15, 1)),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2, padding='same'),
    tf.keras.layers.Dropout(0.5),

    ## - Third Block
    tf.keras.layers.Conv2D(128, kernel_size=(3, 3), strides=2,
                           padding='same', input_shape=(8, 8, 1)),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv2D(128, kernel_size=(3, 3), strides=2,
                           padding='same', input_shape=(8, 8, 1)),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv2D(128, kernel_size=(3, 3), strides=2,
                           padding='same', input_shape=(8, 8, 1)),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2, padding='same'),
    tf.keras.layers.Dropout(0.5),

    ## - Full Connected
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

In [ ]:
top3 = TopKCategoricalAccuracy(k=3, name="top3")
top5 = TopKCategoricalAccuracy(k=5, name="top5")

model_8c2f.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=[Accuracy(), F1Score(), Precision(), Recall(), top3, top5]
)

In [ ]:
model_8c2f.summary()

# -- TensorBoard

In [ ]:
%tensorboard --logdir logs

# -- Training

### V1 8C2F
- No data augmentation
- No batch_size

In [ ]:
mlflow.tensorflow.autolog(log_every_epoch=True,
                          registered_model_name='model_8c2f_v1')

model_8c2f.fit(
    full_ds[0],
    epochs=100,
    validation_data=full_ds[1]
)